In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
df_train = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/gu/gu-train.csv', header=None)
df_train.head()

,0,1
0,entertainment,સાડીમાં જ દેખાતી 'નાગિન 3' ની એક્ટ્રેસનો જુઓ બ...
1,business,"નીરવ મોદીના જેકેટમાં એવુ શું છે ખાસ, જેની કિંમ..."
2,business,નોટ નહી કરાવે ખોટઃશનિવાર અને રવિવાર બેન્કો ખુલ...
3,business,મોબાઇલ : ઇન્ટેક્સ કંપનીએ ઓછી કિંમતમાં લોન્ચ કર...
4,business,"એક જ નિર્ણયથી બદલાઈ ગઈ કિસ્મત, વર્ષભરમાં ફેમિલ..."


In [8]:
Counter(list(df_train[0]))

Counter({'business': 1867, 'entertainment': 2324, 'tech': 1078})

In [9]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/gu/gu-valid.csv', header=None)
df_valid.head()

,0,1
0,business,અમદાવાદઃખેતરમાંથી સીધી હવે કેસર કેરી ગ્રાહકો સ...
1,entertainment,"#MeToo પર રહેમાને તોડી ચુપ્પી, આપ્યું આ નિવેદન"
2,entertainment,IPLમાં પરફોર્મ કરવા માટે રણવીર સિંહે માગી અધધ....
3,entertainment,"બિકીનીમાં બિચ પર જોવા મળી મલાઈકા, તસવીરો શેર ક..."
4,tech,હવે મોબાઇલથી ઘરે બેઠા કમાઇ શકો છો 40 હજાર રૂપિ...


In [10]:
df_test = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/gu/gu-test.csv', header=None)
df_test.head()

,0,1
0,business,"કેપ્ટન વિરાટ કોહલીનો નવો રેકોર્ડ, પ્યૂમા સાથે ..."
1,entertainment,PHOTOS: દેવ દિવાળી ઉજવવા બનારસ પહોંચ્યો અનિલ ક...
2,entertainment,રીતિક રોશને પૂર્વ પત્ની સુઝેન અને બાળકો સાથે ક...
3,entertainment,આ ટીવી એકટ્રેસે પિંક સ્કર્ટમાં કરાવ્યું હોટ ફો...
4,entertainment,નિક સાથે વેકેશન એન્જોય કરવા જંગલમાં નીકળી પ્રિ...


In [11]:
df_train.shape, df_valid.shape, df_test.shape

((5269, 2), (659, 2), (659, 2))

In [12]:
cutoff = int(0.1*len(df_train))

In [13]:
# from inltk.inltk import get_similar_sentences, tokenize
# import warnings
# warnings.filterwarnings("ignore")
# aug_sen = []
# aug_label = []
# for index, row in df_train[len(aug_sen)//2:cutoff].iterrows():
#     augs = get_similar_sentences(row[1], no_of_variations=2, language_code='gu', degree_of_aug=0.3)
#     aug_sen.extend(augs)
#     aug_label.extend([row[0]]*len(augs))
# df_aug = pd.DataFrame({0: aug_label, 1: aug_sen })
# df_aug.head()
# df_aug.to_csv('inltk_headlines_gu_train_aug.csv', index=False)

In [14]:
df_train_aug = pd.read_csv('inltk_headlines_gu_train_aug.csv')
df_train_aug.head()

,0,1
0,entertainment,કપોતમાં પશુપાલન દેખાતી 'નાગાેઝ 2' ની એક્ટ્રેસન...
1,entertainment,કપોતમાં પશુપાલન ક્રોહન ‘નાગિન 3' ની એક્ટ્રેઝનો...
2,business,નીરનાવત નાયપોલના જેકેટમાં એવુ શું છેઃ ખાસ અને ...
3,business,"નારવ મોદીના જેનેકેટમાં એવું શું છે વિશેષ, જેની..."
4,business,નોટ નહિ કરાવે ખોટઃશમૂવાર તેમજ રવિવાર બેન્કો ખુ...


In [15]:
df_train = pd.DataFrame({0: list(df_train[0])[:cutoff] + list(df_train_aug['0'])[:2*cutoff:2], 1: list(df_train[1])[:cutoff] + list(df_train_aug['1'])[:2*cutoff:2]})
df_train.shape

(1052, 2)

In [16]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [17]:
label_cols = [0]

In [18]:
class GujaratiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/gujarati/tokenizer/gujarati_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [19]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/gujarati/tokenizer/gujarati_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(20000)]

In [20]:
itos[:10]

['<unk>', '<s>', '</s>', '.', ',', '▁છે', '▁', '▁અને', 'ની', 'માં']

In [21]:
# 20,000 is the vocab size that we chose in sentencepiece
gujarati_vocab = Vocab(itos)

In [22]:
tokenizer = Tokenizer(tok_func=GujaratiTokenizer, lang='gu')

In [23]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [24]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=gujarati_vocab, label_cols=label_cols)

In [25]:
data_lm.show_batch()

idx,text
0,"ટેક્સ ▁કંપનીએ ▁ઓછી ▁કિંમત માં ▁લોન્ચ ▁કર્યો ▁એક ્વા ▁એ યર ▁2 ▁x x bo s ▁એક ▁જ ▁નિર્ણય થી ▁બદલાઈ ▁ગઈ ▁કિ સ્ મત , ▁વર્ષ ભર માં ▁ફેમિલી ને ▁કરાવ્યો ▁કરોડો નો ▁ફાયદો ▁x x bo s ▁શ્રી દેવી ના ▁રાજકીય ▁સન્માન ▁સાથે ▁અંતિમ ▁સંસ્કાર , <unk> ▁r t i માં ▁થયા ▁ખુલાસા ▁x x bo s ▁લોન ▁ફ્ર ો ડ ▁કેસમાં ▁ચંદ ા ▁કોચ ર ▁અને ▁વીડિયો"
1,"▁સિઝન ▁શરૂ ▁થાય ▁તે ▁પહેલા ▁તેનું ▁આ ▁સિ ક્રેટ ▁જાણ ો ▁x x bo s ▁સલમાન ના ▁ભાઇ ની <unk> ▁e x ▁ગર્લ ફ્રે ન્ડ ▁ઇન્ સ્ટા ગ્રામ ▁પર ▁છ વાઇ , ▁શેર ▁કરી ▁બોલ ્ડ ▁તસવીરો ▁x x bo s ▁આ ઇટ મ ▁ગર્લ ▁નો રા ▁ફતેહ ી એ ▁સેલ િબ ્રેટ ▁કર્યો ▁બર્ થ - ડે , ▁પાર્ટી માં ▁આવ્યા ▁સ્ટાર ્સ ▁x x bo s <unk> ▁p"
2,"▁અહીંયા થી ▁ખરીદ ો ▁બ્રાન્ડ ેડ ▁કપડા ▁x x bo s ▁આશા ▁ભો સ લે ના ▁આ ▁પગલાં ▁બાદ ▁ લતા એ ▁તોડી ▁નાખ્યા ં ▁હતાં ▁તમામ ▁સંબંધ ▁x x bo s ▁આ ▁છે ▁20 18 ના ▁ટોપ ▁10 ▁સ્માર્ટફોન ્સ , ▁ખરીદ શો ▁તો ▁પ સ્તા શો ▁નહીં ▁x x bo s ▁ રિયા ▁ચક્રવર્તી એ ▁શેર ▁કરી ▁હોટ ▁ફોટો શ ૂટ ની ▁તસવીરો , ▁ફે ન્સે ▁કર્યો ▁લાઇ"
3,"s ▁હવે <unk> ▁f ace b ook , <unk> ▁t w it ter થી ▁મોકલ ો ▁પૈસા , ▁શરૂ ▁થઇ ▁નવી ▁સુવિધા ▁x x bo s ▁કો ની ▁સાથે ▁રિ લેશન શિપ માં ▁છે ▁કૈ ટરી ના ▁કૈ ફ ? ▁પોતે ▁જ ▁આપી ▁જાણકારી ▁x x bo s ▁જ ી સ્વા ન ની ▁ખરાબ ▁ક ને ક ટીવ િટી ના ▁પગલે ▁ડિજિટલ ▁પ્રોજેકટ ▁ ખોર ંભ ે ▁ચઢ યો ▁x"
4,"▁ફિચર થી , ▁ઘરે ▁બેઠા ▁આ ▁રીતે ▁કરો ▁પૈસા ▁ટ્રાન્સફર ▁x x bo s ▁રણ વીર ▁સિંહ ▁અને ▁આ લિયા ▁ભટ્ટ ના ▁કિ સિંગ ▁ સીન ▁પર ▁ચાલી ▁કાતર ▁x x bo s ▁ત નુ શ્રી ▁બાદ ▁સ ની ▁લિયો ને ▁જણાવી ▁આપવી તી , ▁મળ્યા ▁હતા ▁આવા ▁પ્રસ્તાવ ▁x x bo s ▁1 ▁જૂન થી ▁બદલ ાશે <unk> ▁p f ▁ઉપાડ ના ▁નિયમો , ▁શું ▁છે ▁નવા ▁નિયમો ?"


In [26]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [27]:
# Loading the pretrained language model on gujarati wikipedia
learn.load('../../../models/gujarati/lm/ULMFiT/third_gu_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (1052 items)
x: LMTextList
▁x x bo s ▁સાડી માં ▁જ ▁દેખાતી ▁' ના ગ િન ▁3 ' ▁ની ▁એક્ટ ્રે સ નો ▁જુઓ ▁બોલ ્ડ ▁અવતાર,▁x x bo s ▁ની ર વ ▁મોદી ના ▁જે કેટ માં ▁એવુ ▁શું ▁છે ▁ખાસ , ▁જેની ▁કિંમત ▁છે ▁આઠ ▁લાખ ▁રૂપિયા,▁x x bo s ▁નોટ ▁નહી ▁કરાવે ▁ખોટ ઃ શ નિ વાર ▁અને ▁રવિવાર ▁બેન્કો ▁ખુલ્લી ▁રહેશે,▁x x bo s ▁મોબાઇલ ▁: ▁ઇન્ ટેક્સ ▁કંપનીએ ▁ઓછી ▁કિંમત માં ▁લોન્ચ ▁કર્યો ▁એક ્વા ▁એ યર ▁2,▁x x bo s ▁એક ▁જ ▁નિર્ણય થી ▁બદલાઈ ▁ગઈ ▁કિ સ્ મત , ▁વર્ષ ભર માં ▁ફેમિલી ને ▁કરાવ્યો ▁કરોડો નો ▁ફાયદો
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (659 items)
x: LMTextList
▁x x bo s ▁અમદાવાદ ઃ ખેત ર માંથી ▁સીધી ▁હવે ▁કેસર ▁કેરી ▁ગ્રાહકો ▁સુધી ▁પહોચ શે , મે યર ે ▁કેરી ▁મહોત્સવ ▁ખુલ્લો ▁મુક્યો,▁x x bo s ▁ <unk> ▁me t o o ▁પર ▁રહેમાન ે ▁તોડી ▁ચુ પ્પી , ▁આપ્યું ▁આ ▁નિવેદન,▁x x bo s <unk> <unk> l માં ▁પર ફોર્મ ▁કરવા ▁માટે ▁રણ વીર ▁સિંહ ે ▁માગી ▁અધ ધ . . . ર કમ,▁x x bo s ▁બિ કી ની માં ▁બિ ચ ▁પર ▁જોવા ▁મળી ▁મ લાઈ કા , ▁તસવીરો ▁શેર ▁કરતાં ▁જ ▁થઈ ▁ટ્રો લ,▁x x bo s ▁હવે ▁મ

In [28]:
learn.freeze()

In [29]:
learn.fit_one_cycle(1, 1e-1)

epoch,train_loss,valid_loss,accuracy,time
0,6.289598,5.833225,0.244699,00:00


In [30]:
learn.unfreeze()

In [31]:
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.353076,5.114670,0.286161,00:00
1,3.761254,5.297334,0.293248,00:00
2,3.259233,5.349907,0.296819,00:00


In [32]:
learn.predict('શાહરૂખ ખાન સાથે',n_words=10)

'શાહરૂખ ખાન સાથે ▁મિત્રતા ▁સાથે ▁પુસ્તકો ▁તેમનાં ▁ખુલાસા ▁કોશિષ ▁કર્યાં ▁ત્રણ ▁શિક્ષકો ,'

In [33]:
learn.save_encoder('fine_tuned_enc')

In [34]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=gujarati_vocab, label_cols=label_cols, bs=64)

In [35]:
data_clas.show_batch()

text,target
▁x x bo s ▁ <unk> <unk> h o <unk> <unk> ▁group g : ▁તબ ૈ હુ ર ની ▁બર્ થ ▁ડે ▁સેલ િબ ્રેટ ▁કરવા ં ▁સૈ ફ ▁તેમજ ▁કરી શિ ના ▁પહોચ ્યા ▁કેપ ▁ટાઉન,entertainment
"▁x x bo s ▁રે ▁બે કારી ! ▁સ્વ િ પર ની ▁14 ▁જગ્યા ▁માટે <unk> ▁m . t e ch , <unk> ▁b . t e ch , <unk> ▁m b a ▁ઉમેદવારો એ ▁અરજી ▁કરી",business
"▁x x bo s ▁ફ્રે ▁બે કારી ! ▁સ્વ િ ▁ગોલંદાજ ના ▁13 ▁જગ્યાઓ ▁માટેની <unk> ▁m . t e ch , <unk> . t e ch ; <unk> ▁m r a ▁ઉમેદવારો એ ▁અરજી ▁કરી",business
▁x x bo s ▁પ x x un k 10 ▁રદિ ear <unk> <unk> h all en ious ▁કાન્હ ાઇ કી ▁રફ ોરા એ ▁એવી ▁તે ▁શું ▁ભૂલ ▁કરીને ▁કે ▁ફે ન્સે ▁ઉડાવ ી ▁મજાક ?,entertainment
"▁x x bo s ▁coriander e <unk> <unk> n g ▁ <unk> <unk> <unk> <unk> ▁ડિસ્લેક્સીયા n : ▁કેન્સર ગ્રસ્ત ▁બાળક ને ▁મળવા ▁પહોચ ્યા ▁સલમાન , ▁જુઓ <unk> ▁v i d ▁ચિત્તભ્રમણા o",entertainment


In [36]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [37]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [38]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (1052 items)
x: TextList
▁x x bo s ▁સાડી માં ▁જ ▁દેખાતી ▁' ના ગ િન ▁3 ' ▁ની ▁એક્ટ ્રે સ નો ▁જુઓ ▁બોલ ્ડ ▁અવતાર,▁x x bo s ▁ની ર વ ▁મોદી ના ▁જે કેટ માં ▁એવુ ▁શું ▁છે ▁ખાસ , ▁જેની ▁કિંમત ▁છે ▁આઠ ▁લાખ ▁રૂપિયા,▁x x bo s ▁નોટ ▁નહી ▁કરાવે ▁ખોટ ઃ શ નિ વાર ▁અને ▁રવિવાર ▁બેન્કો ▁ખુલ્લી ▁રહેશે,▁x x bo s ▁મોબાઇલ ▁: ▁ઇન્ ટેક્સ ▁કંપનીએ ▁ઓછી ▁કિંમત માં ▁લોન્ચ ▁કર્યો ▁એક ્વા ▁એ યર ▁2,▁x x bo s ▁એક ▁જ ▁નિર્ણય થી ▁બદલાઈ ▁ગઈ ▁કિ સ્ મત , ▁વર્ષ ભર માં ▁ફેમિલી ને ▁કરાવ્યો ▁કરોડો નો ▁ફાયદો
y: CategoryList
entertainment,business,business,business,business
Path: .;

Valid: LabelList (659 items)
x: TextList
▁x x bo s ▁અમદાવાદ ઃ ખેત ર માંથી ▁સીધી ▁હવે ▁કેસર ▁કેરી ▁ગ્રાહકો ▁સુધી ▁પહોચ શે , મે યર ે ▁કેરી ▁મહોત્સવ ▁ખુલ્લો ▁મુક્યો,▁x x bo s ▁ <unk> ▁me t o o ▁પર ▁રહેમાન ે ▁તોડી ▁ચુ પ્પી , ▁આપ્યું ▁આ ▁નિવેદન,▁x x bo s <unk> <unk> l માં ▁પર ફોર્મ ▁કરવા ▁માટે ▁રણ વીર ▁સિંહ ે ▁માગી ▁અધ ધ . . . ર કમ,▁x x bo s ▁બિ કી ની માં ▁બિ ચ ▁પર ▁જોવા ▁મળી ▁મ લાઈ કા , ▁તસવીરો ▁શેર 

In [39]:
learn.freeze()

In [40]:
learn.loss_func.func

CrossEntropyLoss()

In [41]:
mcc = MatthewsCorreff()

In [42]:
learn.metrics = [mcc, accuracy]

In [43]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.728411,0.697247,0.592365,0.742033,00:01


In [44]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.489638,0.580840,0.591518,0.738998,00:01


In [45]:
learn.save('second-full')

In [46]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (1052 items)
x: TextList
▁x x bo s ▁સાડી માં ▁જ ▁દેખાતી ▁' ના ગ િન ▁3 ' ▁ની ▁એક્ટ ્રે સ નો ▁જુઓ ▁બોલ ્ડ ▁અવતાર,▁x x bo s ▁ની ર વ ▁મોદી ના ▁જે કેટ માં ▁એવુ ▁શું ▁છે ▁ખાસ , ▁જેની ▁કિંમત ▁છે ▁આઠ ▁લાખ ▁રૂપિયા,▁x x bo s ▁નોટ ▁નહી ▁કરાવે ▁ખોટ ઃ શ નિ વાર ▁અને ▁રવિવાર ▁બેન્કો ▁ખુલ્લી ▁રહેશે,▁x x bo s ▁મોબાઇલ ▁: ▁ઇન્ ટેક્સ ▁કંપનીએ ▁ઓછી ▁કિંમત માં ▁લોન્ચ ▁કર્યો ▁એક ્વા ▁એ યર ▁2,▁x x bo s ▁એક ▁જ ▁નિર્ણય થી ▁બદલાઈ ▁ગઈ ▁કિ સ્ મત , ▁વર્ષ ભર માં ▁ફેમિલી ને ▁કરાવ્યો ▁કરોડો નો ▁ફાયદો
y: CategoryList
entertainment,business,business,business,business
Path: .;

Valid: LabelList (659 items)
x: TextList
▁x x bo s ▁અમદાવાદ ઃ ખેત ર માંથી ▁સીધી ▁હવે ▁કેસર ▁કેરી ▁ગ્રાહકો ▁સુધી ▁પહોચ શે , મે યર ે ▁કેરી ▁મહોત્સવ ▁ખુલ્લો ▁મુક્યો,▁x x bo s ▁ <unk> ▁me t o o ▁પર ▁રહેમાન ે ▁તોડી ▁ચુ પ્પી , ▁આપ્યું ▁આ ▁નિવેદન,▁x x bo s <unk> <unk> l માં ▁પર ફોર્મ ▁કરવા ▁માટે ▁રણ વીર ▁સિંહ ે ▁માગી ▁અધ ધ . . . ર કમ,▁x x bo s ▁બિ કી ની માં ▁બિ ચ ▁પર ▁જોવા ▁મળી ▁મ લાઈ કા , ▁તસવીરો ▁શેર 

In [47]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-2, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.277835,0.577685,0.640092,0.769347,00:01
1,0.197446,0.866296,0.576505,0.725341,00:01
2,0.190956,0.764223,0.660308,0.779970,00:01
3,0.168238,0.665715,0.661455,0.787557,00:01
4,0.123636,0.608131,0.688944,0.804249,00:01


Better model found at epoch 0 with accuracy value: 0.7693474888801575.
Better model found at epoch 2 with accuracy value: 0.7799696326255798.
Better model found at epoch 3 with accuracy value: 0.7875568866729736.
Better model found at epoch 4 with accuracy value: 0.8042488694190979.


In [48]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (1052 items)
x: TextList
▁x x bo s ▁સાડી માં ▁જ ▁દેખાતી ▁' ના ગ િન ▁3 ' ▁ની ▁એક્ટ ્રે સ નો ▁જુઓ ▁બોલ ્ડ ▁અવતાર,▁x x bo s ▁ની ર વ ▁મોદી ના ▁જે કેટ માં ▁એવુ ▁શું ▁છે ▁ખાસ , ▁જેની ▁કિંમત ▁છે ▁આઠ ▁લાખ ▁રૂપિયા,▁x x bo s ▁નોટ ▁નહી ▁કરાવે ▁ખોટ ઃ શ નિ વાર ▁અને ▁રવિવાર ▁બેન્કો ▁ખુલ્લી ▁રહેશે,▁x x bo s ▁મોબાઇલ ▁: ▁ઇન્ ટેક્સ ▁કંપનીએ ▁ઓછી ▁કિંમત માં ▁લોન્ચ ▁કર્યો ▁એક ્વા ▁એ યર ▁2,▁x x bo s ▁એક ▁જ ▁નિર્ણય થી ▁બદલાઈ ▁ગઈ ▁કિ સ્ મત , ▁વર્ષ ભર માં ▁ફેમિલી ને ▁કરાવ્યો ▁કરોડો નો ▁ફાયદો
y: CategoryList
entertainment,business,business,business,business
Path: .;

Valid: LabelList (659 items)
x: TextList
▁x x bo s ▁અમદાવાદ ઃ ખેત ર માંથી ▁સીધી ▁હવે ▁કેસર ▁કેરી ▁ગ્રાહકો ▁સુધી ▁પહોચ શે , મે યર ે ▁કેરી ▁મહોત્સવ ▁ખુલ્લો ▁મુક્યો,▁x x bo s ▁ <unk> ▁me t o o ▁પર ▁રહેમાન ે ▁તોડી ▁ચુ પ્પી , ▁આપ્યું ▁આ ▁નિવેદન,▁x x bo s <unk> <unk> l માં ▁પર ફોર્મ ▁કરવા ▁માટે ▁રણ વીર ▁સિંહ ે ▁માગી ▁અધ ધ . . . ર કમ,▁x x bo s ▁બિ કી ની માં ▁બિ ચ ▁પર ▁જોવા ▁મળી ▁મ લાઈ કા , ▁તસવીરો ▁શેર 

In [49]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,entertainment,business,tech
0,"કેપ્ટન વિરાટ કોહલીનો નવો રેકોર્ડ, પ્યૂમા સાથે ...",business,entertainment,0.988475,0.00959925,0.00192529
1,PHOTOS: દેવ દિવાળી ઉજવવા બનારસ પહોંચ્યો અનિલ ક...,entertainment,entertainment,0.927368,0.0596439,0.0129882
2,રીતિક રોશને પૂર્વ પત્ની સુઝેન અને બાળકો સાથે ક...,entertainment,entertainment,0.992044,0.00594941,0.00200664
3,આ ટીવી એકટ્રેસે પિંક સ્કર્ટમાં કરાવ્યું હોટ ફો...,entertainment,entertainment,0.998341,0.000843149,0.000815976
4,નિક સાથે વેકેશન એન્જોય કરવા જંગલમાં નીકળી પ્રિ...,entertainment,entertainment,0.993775,0.00442895,0.00179621


In [50]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.8103186646433991

In [51]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.7043929258370422

In [52]:
df_result.to_csv('inltk_headlines_gu_with_aug.csv', index=False)